In [14]:
!pip install torch torchvision
!pip install ray
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torchvision.models as models
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total

In [48]:
def data_loader(batch_size=64):
  
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

  training_data = torchvision.datasets.STL10("/content", 
                                             split="train",
                                              transform = transforms.Compose([
                                                                              transforms.ToTensor(),  
                                                                              normalize,            
                                                                              ]), 
                                              download=True)
  
  val_data = torchvision.datasets.STL10("/content", 
                                        split="train",
                                              transform = transforms.Compose([transforms.ToTensor(),   normalize,              
                                                                              ]), 
                                              download=True)

  testing_data = torchvision.datasets.STL10("/content", 
                                            split="test",
                                              transform = transforms.Compose([transforms.ToTensor(),     normalize,                   
                                                                              ]), 
                                              download=True)
  #mean, std = calc_normalization(train_dl)
  num_train = len(training_data)
  indices = list(range(num_train))
  split = 100 #45k/5k train/val split
  train_idx, valid_idx = indices[split:], indices[:split]
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)
  training_data_loader = torch.utils.data.DataLoader(training_data, batch_size, sampler=train_sampler, shuffle=False)
  val_data_loader = torch.utils.data.DataLoader(val_data, batch_size, sampler=valid_sampler, shuffle=False)
  testing_data_loader = torch.utils.data.DataLoader(testing_data, batch_size, shuffle=True)
  return training_data_loader, val_data_loader, testing_data_loader

In [26]:
data_loader()

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting /content/stl10_binary.tar.gz to /content
Files already downloaded and verified
Files already downloaded and verified


(<torch.utils.data.dataloader.DataLoader at 0x7f6540976090>,
 <torch.utils.data.dataloader.DataLoader at 0x7f653fbe5390>)

In [49]:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/densenet.py
import math

class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out


class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out


class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def DenseNet121(blocks=[6,12,24,16], k=12):
    return DenseNet(Bottleneck, blocks, growth_rate=k)

In [50]:
def test(net, testloader, device, epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    test_steps = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            test_steps += 1
    print("epoch {} test_loss {} test_steps {} test_acc {}".format(epoch, test_loss, test_steps, correct / total))
    return correct/total

In [55]:
import matplotlib.pyplot as plt
def train_dense(w_decay=0.0001, momentum_arg=0.9, batch_size=64):
  net = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
  
  trainloader, valloader, testloader = data_loader(batch_size)
  classes = ('airplane', 'car', 'bird', 'cat', 'deer',
           'dog', 'monkey', 'horse', 'ship', 'truck')
  
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          net = nn.DataParallel(net)
  net.to(device)

  def _lr_lambda(current_step):
      """
      _lr_lambda returns a multiplicative factor given an interger parameter epochs.
      """
      if current_step < 25000:
          _lr = 1
      elif current_step < 37500:
          _lr = .1
      else:
          _lr = .01
      return _lr

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=momentum_arg, weight_decay=w_decay)
  scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, _lr_lambda, last_epoch=-1, verbose=True)
  
  train_accs = []
  val_accs = []
  test_accs = []
  epochs = []

  for epoch in range(20):  # loop over the dataset multiple times
        # if epoch % 5 == 4:
        #   fig = plt.figure()
        #   plt.plot(epochs, test_accs)
        #   plt.xlabel('epochs') 
        #   plt.ylabel('test error rate') 
        #   plt.title("epoch: {}".format(epoch))
        #   fig.savefig('temp{}.png'.format(epoch), dpi=fig.dpi)
        #   # files.download("temp{}.png") 

        #   fig = plt.figure()
        #   plt.plot(epochs, train_accs)
        #   plt.xlabel('epochs') 
        #   plt.ylabel('train error rate') 
        #   plt.title("epoch:0 - {}".format(epoch))
        #   fig.savefig('temptrain{}.png'.format(epoch), dpi=fig.dpi)
        #   #files.download("temp{}.png") 

        #   fig = plt.figure()
        #   plt.plot(epochs, val_accs)
        #   plt.xlabel('epochs') 
        #   plt.ylabel('validation error rate') 
        #   plt.title("epoch:0 - {}".format(epoch))
        #   fig.savefig('tempval{}.png'.format(epoch), dpi=fig.dpi)
        #   #files.download("tempval{}.png") 
          

        running_loss = 0.0
        epoch_steps = 0
        train_loss = 0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            train_loss += loss.item()
            epoch_steps += 1
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            if i % 100 == 99:  # print every 100 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
                # print("{}".format(correct/total))
            running_loss = 0.0
        train_accs.append(correct/total)
           
        
        # # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
        print("epoch {} val_loss {} val_steps {} val_acc {}".format(epoch, val_loss, val_steps, correct / total))
        val_accs.append(1 - correct/total)
        epochs.append(epoch)
        
        # tacc = test(net, testloader, device, epoch)
        # test_accs.append(1 - tacc)
        #print("Finished 1 eps")




  #final test error rate
  test(net, testloader, device, epoch)
  print("Finished Training")
  return net, train_accs, val_accs, test_accs, epochs


In [56]:
net, train_accs, val_accs, test_accs, epochs = train_dense()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Adjusting learning rate of group 0 to 1.0000e-01.
epoch 0 val_loss 4.087257146835327 val_steps 2 val_acc 0.25
epoch 1 val_loss 3.8590670824050903 val_steps 2 val_acc 0.23
epoch 2 val_loss 3.172651171684265 val_steps 2 val_acc 0.38
epoch 3 val_loss 2.7768797874450684 val_steps 2 val_acc 0.43
epoch 4 val_loss 2.739535093307495 val_steps 2 val_acc 0.48
epoch 5 val_loss 2.7149510383605957 val_steps 2 val_acc 0.56
epoch 6 val_loss 2.3250298500061035 val_steps 2 val_acc 0.58
epoch 7 val_loss 2.666993260383606 val_steps 2 val_acc 0.49
epoch 8 val_loss 2.4045073986053467 val_steps 2 val_acc 0.6
epoch 9 val_loss 2.020582616329193 val_steps 2 val_acc 0.61
epoch 10 val_loss 2.4555119276046753 val_steps 2 val_acc 0.6
epoch 11 val_loss 2.4763827323913574 val_steps 2 val_acc 0.59
epoch 12 val_loss 2.2041577100753784 val_steps 2 val_acc 0.64
epoch 13 val_loss 2.14738667011261 val_steps 2 